In [9]:
import pandas as pd

In [45]:
from trading_bot.agent import Agent
model_name = 'model_double-dqn_GOOG_50_30'
test_stock = 'data/BTCUSDT_TESTING.csv'
window_size = 10
debug = True
agent = Agent(window_size, pretrained=True, model_name=model_name)

In [46]:
df = pd.read_csv(test_stock)
df = df[['Date', 'Close']]
df = df.rename(columns={'Close': 'actual', 'Date': 'date'})
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates
df

,date,actual
0,2019-12-15,7118.59
1,2019-12-16,6891.72
2,2019-12-17,6623.82
3,2019-12-18,7277.83
4,2019-12-19,7150.30
...,...,...
134,2020-04-27,7774.62
135,2020-04-28,7738.98
136,2020-04-29,8778.57
137,2020-04-30,8620.00


In [47]:
import logging
import coloredlogs

from trading_bot.utils import show_eval_result, switch_k_backend_device, get_stock_data
from trading_bot.methods import evaluate_model

coloredlogs.install(level='DEBUG')
switch_k_backend_device()

test_data = get_stock_data(test_stock)
initial_offset = test_data[1] - test_data[0]

test_result, history = evaluate_model(agent, test_data, window_size, debug)
show_eval_result(model_name, test_result, initial_offset)

2020-05-27 00:30:42 darkdevil root[9129] DEBUG switching to TensorFlow for CPU


[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7118.59
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $6891.72
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $6623.82
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Sell at: $7277.83 | Position: +$159.24
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7150.30
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Sell at: $7187.83 | Position: +$296.11
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7132.75
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7501.44
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7317.09
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7255.77
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7204.63
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7202.00
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Sell at: $7254.74 | Position: +$630.92
2020-05-27 00:30:53 darkdevil root[9129] DEBUG Buy at: $7316.14
2020-05-27 00:30:53 darkdevil root[912

In [13]:
import lxml.html as lh
import time
import urllib.request
import argparse
import urllib.request, urllib.parse, urllib.error
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import json
import ssl
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)

In [14]:
tz = pytz.timezone('Asia/Kolkata')
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [48]:
def Real(url):
    uh = urllib.request.urlopen(url, context=ctx)
    data = uh.read().decode()
    info = json.loads(data)
    live = info['price'];
    time.sleep(1)
    return float(live)

In [52]:
def evaluate_model(agent, data, window_size, debug):
    url = "https://api.binance.com/api/v1/ticker/price?symbol=BTCUSDT"
    count = 0
    while count < window_size:
        live = Real(url)
        data.append(live)
        count += 1
    total_profit = 0
    history = []
    agent.inventory = []
    state = get_state(data, 0, window_size + 1)
    for t in range(100):   
        live = Real(url)
        data.append(live)
        reward = 0
        next_state = get_state(data, t + 1, window_size + 1)
        
        # select an action
        action = agent.act(state, is_eval=True)

        # BUY
        if action == 1:
            agent.inventory.append(data[t])
            history.append((data[t], "BUY"))
            if debug:
                logging.debug("Buy at: {}".format(format_currency(data[t])))
        
        # SELL
        elif action == 2 and len(agent.inventory) > 0:
            bought_price = agent.inventory.pop(0)
            delta = data[t] - bought_price
            reward = delta #max(delta, 0)
            total_profit += delta

            history.append((data[t], "SELL"))
            if debug:
                logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
        else:
            history.append((data[t], "HOLD"))

        done = (t == (100 - 1))
        agent.memory.append((state, action, reward, next_state, done))

        state = next_state
        if done:
            return total_profit, history

In [53]:
def main(args):
    price = []
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    model_name='model_double-dqn_GOOG_50_30'

    agent = Agent(window_size, pretrained=True, model_name=model_name)
    profit, history = evaluate_model(agent, price, window_size, debug=True)
    print("Profit:", profit)
    buys = sells = holds = 0
    for i in history:
        if i[1] == "BUY":
            buys += 1
        elif i[1] == "SELL":
            sells += 1
        elif i[1] == "HOLD":
            holds += 1
    print("BUYS Percentage:", (buys/len(history)) * 100)
    print("SELLS Percentage:", (sells/len(history)) * 100)
    print("HOLDS Percentage:", (holds/len(history)) * 100)

In [54]:
coloredlogs.install(level="DEBUG")
switch_k_backend_device()
args = argparse.Namespace(ticker='BTCUSDT')
try:
    main(args)
except KeyboardInterrupt:
    print("Aborted")  

2020-05-27 00:34:31 darkdevil root[9129] DEBUG switching to TensorFlow for CPU
2020-05-27 00:34:46 darkdevil root[9129] DEBUG Buy at: $8800.51
2020-05-27 00:34:48 darkdevil root[9129] DEBUG Buy at: $8800.51
2020-05-27 00:34:49 darkdevil root[9129] DEBUG Buy at: $8801.19
2020-05-27 00:34:51 darkdevil root[9129] DEBUG Buy at: $8801.19
2020-05-27 00:34:52 darkdevil root[9129] DEBUG Buy at: $8801.19
2020-05-27 00:34:53 darkdevil root[9129] DEBUG Sell at: $8801.19 | Position: +$0.68
2020-05-27 00:34:54 darkdevil root[9129] DEBUG Buy at: $8800.91
2020-05-27 00:34:56 darkdevil root[9129] DEBUG Sell at: $8800.83 | Position: +$0.32
2020-05-27 00:34:57 darkdevil root[9129] DEBUG Sell at: $8800.67 | Position: -$0.52
2020-05-27 00:34:58 darkdevil root[9129] DEBUG Buy at: $8800.67
2020-05-27 00:35:00 darkdevil root[9129] DEBUG Sell at: $8800.54 | Position: -$0.65
2020-05-27 00:35:01 darkdevil root[9129] DEBUG Sell at: $8800.54 | Position: -$0.65
2020-05-27 00:35:02 darkdevil root[9129] DEBUG Buy at

Profit: 1.7100000000009459
BUYS Percentage: 42.0
SELLS Percentage: 42.0
HOLDS Percentage: 16.0
